# Demo the Text-to-Speech module

In [1]:
import glados.tts as tts
import sounddevice as sd

In [2]:
# Instantiate the TTS engine
glados_tts = tts.TTSEngine()

Loading libespeak-ng.so.1


/home/michael/miniconda3/envs/glados/lib/python3.11/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


In [3]:
# Generate the audio.
# Glados is spelt incorrectly on purpose to make the pronunciation more accurate.
audio = glados_tts.generate_speech_audio("Hello, my name is Glados. I am an AI created by Aperture Science.")

# Play the audio
sd.play(audio, tts.RATE)

# Demo the Automatic speech recogntion system
This will detect and transcribe your voice. In this demo, it will then get GlaDOS to repeat back to you what was heard.

In [4]:
import glados.voice_recognition as vr

In [5]:
def say_text(text: str):
    """Say text using text-to-speech engine
    """
    audio = glados_tts.generate_speech_audio(text)
    sd.play(audio, tts.RATE)
    sd.wait()

# Instantiate VoiceRecognition class with the say_text function
demo = vr.VoiceRecognition(function=say_text)

# Start the demo
demo.start()

2024-06-01 19:15:57.689050525 [W:onnxruntime:, graph.cc:3593 CleanUnusedInitializersAndNodeArgs] Removing initializer '131'. It is not used by any node and should be removed from the model.
2024-06-01 19:15:57.689080169 [W:onnxruntime:, graph.cc:3593 CleanUnusedInitializersAndNodeArgs] Removing initializer '136'. It is not used by any node and should be removed from the model.
2024-06-01 19:15:57.689087082 [W:onnxruntime:, graph.cc:3593 CleanUnusedInitializersAndNodeArgs] Removing initializer '139'. It is not used by any node and should be removed from the model.
2024-06-01 19:15:57.689093868 [W:onnxruntime:, graph.cc:3593 CleanUnusedInitializersAndNodeArgs] Removing initializer '140'. It is not used by any node and should be removed from the model.
2024-06-01 19:15:57.689101326 [W:onnxruntime:, graph.cc:3593 CleanUnusedInitializersAndNodeArgs] Removing initializer '134'. It is not used by any node and should be removed from the model.
2024-06-01 19:15:57.689186990 [W:onnxruntime:, gra

KeyboardInterrupt: 

In [ ]:
say_text("Goodbye")

In [ ]:
from pathlib import Path
Path.cwd()

In [ ]:
from glados import vad
from pathlib import Path
from queue import Queue
import sounddevice as sd

VAD_MODEL = "silero_vad.onnx"

PAUSE_TIME = 0.05  # Time to wait between processing loops
SAMPLE_RATE = 16000  # Sample rate for input stream
VAD_SIZE = 50  # Milliseconds of sample for Voice Activity Detection (VAD)
VAD_THRESHOLD = 0.9  # Threshold for VAD detection
BUFFER_SIZE = 600  # Milliseconds of buffer before VAD detection
PAUSE_LIMIT = 400  # Milliseconds of pause allowed before processing


vad_model = vad.VAD(model_path=str(Path.cwd() / "models" / VAD_MODEL))

q = Queue()

def audio_callback(indata, frames, time, status):
    data = indata.copy()
    data = data.squeeze()
    conf = vad_model.process_chunk(data)
    # vad_confidence = conf > VAD_THRESHOLD
    q.put(conf)
    print(conf)


input_stream = sd.InputStream(
            samplerate=SAMPLE_RATE,
            channels=1,
            callback=audio_callback,
            blocksize=int(SAMPLE_RATE * VAD_SIZE / 1000),
        )

In [ ]:
input_stream.start()

In [ ]:
from main import Glados
demo = Glados(wake_word=None)
demo.start()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import time
import queue

fig, ax = plt.subplots()
ax.set_ylim(0, 1)       # Adjust the y-axis to accommodate all data points

x, y = [], []
line, = ax.plot(x, y, 'r-')  # 'r-' is for a red line

data_queue = queue.Queue()

def feed_data():
    for _ in range(100):  # Simulate 100 data points
        time.sleep(0.1)  # Simulate time delay
        data_queue.put(np.random.rand())
    data_queue.put(None)  # Signal to stop the generator


def data_gen():
    t = 0
    while True:
        t += 1
        value = q.get()
        yield t, value  # This would be replaced with your callback function

def update(data):
    t, value = data
    x.append(t)
    y.append(value)
    line.set_data(x, y)
    ax.set_xlim(max(0, t-50), t+50)  # Adjust the x-axis to show a moving window
    return line,

ani = FuncAnimation(fig, update, frames=data_gen, blit=True)

plt.show()


In [ ]:
input_stream.stop()